In [61]:
import k3d
import numpy as np
from scipy import stats
import scipy.optimize as optimize

import sys
sys.path.insert(0, "..")

from Geometry.HALL import HALL
from Geometry.SM import SM

In [88]:
# Analyze the Sept 10 2024 surveys of the bottom and top supermodules in Bldg 157 (without cameras)
# The mPMTs measured using a long extension - whose length is to be determined

# Define the survey area as a Hall that contains the supermodules.

class Room(HALL):
    # define a room containing the two supermodules (Bldg 157)

    # select the encap supermodule type to be used in the survey by setting the index
    super_module = {0:'bottom', 1:'top'}[1]

    room_sms = []
    # one supermodule in the room - its position and orientation are not well known
    # move the supermodule up by +/- 30 mm to match more closely the survey data as a starting point
    room_sms.append({'name': super_module, 'kind': super_module,
                    'loc': [0., 0., -30.],
                    'loc_sig': [100.0, 100.0, 30.0],
                    'rot_axes': 'ZYX',
                    'rot_angles': [0., 0., 0.],
                    'rot_angles_sig': [0.02, 0.02, 0.04]})

    HALL.sms_design['room'] = room_sms

In [89]:
# create the supermodules and place them in the room:

my_room = Room('my_room', kind='room', device_type=SM)

In [90]:
# Survey data: dictionary mpmt:c#:[x,y,z] in meters from survey group
# The survey coordinate systems for the endcaps follow the WCTE conventions (apart from the origin)

#bottom survey missing 11c2, 13c1
bottom_survey_data = {
    0:{1:[0.19633,0.19907,-0.00531],2:[0.19840,-0.19610,-0.00628],3:[-0.19668,0.19684,-0.00574],4:[-0.19504,-0.19801,-0.00708]},
    1:{1:[0.77643,0.19933,-0.00081],2:[0.77862,-0.19605,-0.00190],3:[0.38301,0.19676,-0.00425],4:[0.38497,-0.19822,-0.00513]},
    2:{1:[0.77884,0.77646,-0.00041],2:[0.77656,0.38046,-0.00056],3:[0.38401,0.77724,-0.00355],4:[0.38286,0.38257,-0.00400]},
    3:{1:[-0.19700,0.38157,-0.00545],2:[-0.19668,0.77662,-0.00490],3:[0.19609,0.38030,-0.00537],4:[0.19741,0.77603,-0.00482]},
    4:{1:[-0.77435,0.38145,-0.00242],2:[-0.77509,0.77477,-0.00112],3:[-0.38215,0.38064,-0.00480],4:[-0.38176,0.77404,-0.00430]},
    5:{1:[-0.77469,-0.19898,-0.00479],2:[-0.77702,0.19498,-0.00317],3:[-0.38164,-0.19834,-0.00650],4:[-0.38331,0.19599,-0.00527]},
    6:{1:[-0.77548,-0.77778,-0.00302],2:[-0.77685,-0.38311,-0.00485],3:[-0.38130,-0.77552,-0.00474],4:[-0.38341,-0.38115,-0.00651]},
    7:{1:[-0.19509,-0.77918,-0.00456],2:[-0.19710,-0.38458,-0.00649],3:[0.19764,-0.77715,-0.00350],4:[0.19555,-0.38298,-0.00568]},
    8:{1:[0.77654,-0.38117,-0.00151],2:[0.77660,-0.77629,0.00077],3:[0.38381,-0.38151,-0.00470],4:[0.38396,-0.77581,-0.00262]},
    9:{1:[1.35823,0.19742,0.00895],2:[1.35660,-0.19742,0.00730],3:[0.96578,0.19843,0.00231],4:[0.96284,-0.19458,0.00100]},
    10:{1:[1.35919,0.77375,0.00809],2:[1.35931,0.37968,0.00900],3:[0.96478,0.77511,0.00329],4:[0.96561,0.38061,0.00306]},
    11:{1:[0.77821,1.35644,-0.00217],3:[0.38438,1.35576,-0.00557],4:[0.38424,0.96165,-0.00405]},
    12:{1:[0.19983,1.35732,0.00340],2:[0.19990,0.96099,0.00498],3:[-0.19587,1.35669,0.00340],4:[-0.19587,0.96052,0.00488]},
    13:{2:[-0.77507,1.35396,-0.00223],3:[-0.38110,0.96038,-0.00482],4:[-0.38151,1.35484,-0.00561],},
    14:{1:[-1.35725,0.37871,0.01455],2:[-1.35729,0.77506,0.01443],3:[-0.96183,0.37892,0.00949],4:[-0.96161,0.77469,0.00948]},
    15:{1:[-1.35806,-0.19744,0.00151],2:[-1.35677,0.19744,0.00419],3:[-0.96365,-0.19959,-0.00324],4:[-0.96250,0.19446,-0.00136]},
    16:{1:[-1.35490,-0.78169,0.01301],2:[-1.35710,-0.38533,0.01122],3:[-0.95925,-0.77965,0.00845],4:[-0.96145,-0.38384,0.00658]},
    17:{1:[-0.77479,-1.35591,-0.00469],2:[-0.77378,-0.96141,-0.00259],3:[-0.38126,-1.35588,-0.00638],4:[-0.38043,-0.96203,-0.00434]},
    18:{1:[-0.19623,-1.35452,0.00359],2:[-0.19656,-0.95862,0.00576],3:[0.20011,-1.35454,0.00482],4:[0.19921,-0.95843,0.00671]},
    19:{1:[0.77628,-0.96041,0.00244],2:[0.77744,-1.35405,0.00057],3:[0.38426,-0.96204,-0.00209],4:[0.38507,-1.35475,-0.00388]},
    20:{1:[1.35738,-0.38175,0.00734],2:[1.35694,-0.77567,0.00970],3:[0.96625,-0.38191,0.00124],4:[0.96426,-0.77492,0.00359]}
}

top_survey_data = {
    86:{1:[0.77278,-0.19588,-0.00395],2:[0.77300,0.19690,-0.00413],3:[0.38059,-0.19460,-0.00700],4:[0.38072,0.19785,-0.00723]},
    87:{1:[0.77284,0.38029,-0.00355],2:[0.77194,0.77314,-0.00082],3:[0.38049,0.38228,-0.00657],4:[0.38178,0.77491,-0.00330]},
    88:{1:[-0.19668,0.77315,-0.00351],2:[-0.19600,0.38044,-0.00657],3:[0.19433,0.77344,-0.00411],4:[0.19654,0.38157,-0.00741]},
    89:{1:[-0.77328,0.77397,0.00084],2:[-0.77368,0.38184,-0.00208],3:[-0.38256,0.77431,-0.00281],4:[-0.38181,0.38064,-0.00551]},
    90:{1:[-0.77359,0.19665,-0.00279],2:[-0.77403,-0.19556,-0.00406],3:[-0.38305,0.19622,-0.00631],4:[-0.38334,-0.19474,-0.00695]},
    92:{1:[-0.19779,-0.37885,-0.00687],2:[-0.19699,-0.77165,-0.00398],3:[0.19429,-0.37853,-0.00681],4:[0.19526,-0.77188,-0.00388]},
    93:{1:[0.77058,-0.77189,-0.00004],2:[0.77237,-0.37862,-0.00346],3:[0.37953,-0.77052,-0.00348],4:[0.37962,-0.37798,-0.00642]},
    94:{1:[1.34929,-0.19660,0.00430],2:[1.35046,0.19660,0.00404],3:[0.95708,-0.19484,-0.00183],4:[0.95796,0.19765,-0.00195]},
    95:{1:[1.35146,0.38186,0.00414],2:[1.34976,0.77492,0.00582],3:[0.95934,0.38063,-0.00152],4:[0.95904,0.77279,0.00111]},
    96:{1:[0.77053,0.95847,0.00115],2:[0.77292,1.35110,0.00654],3:[0.38131,0.95792,-0.00112],4:[0.37947,1.35029,0.00581]},
    97:{1:[-0.19666,1.34966,0.00680],2:[-0.19650,0.95746,-0.00132],3:[0.19508,1.35024,0.00582],4:[0.19503,0.95796,-0.00141]},
    98:{1:[-0.77394,1.35104,0.01057],2:[-0.77434,0.96136,0.00321],3:[-0.38216,1.34978,0.00774],4:[-0.38246,0.95836,-0.00075]},
    100:{1:[-1.34972,0.19618,0.00604],2:[-1.35004,-0.19618,0.00436],3:[-0.95954,0.19539,-0.00043],4:[-0.95953,-0.19610,-0.00194]},
    101:{1:[-1.35234,-0.37987,0.00453],2:[-1.35199,-0.77395,0.00691],3:[-0.96021,-0.38057,-0.00159],4:[-0.96096,-0.77342,0.00162]},
    102:{1:[-0.77303,-0.95934,0.00154],2:[-0.77378,-1.35271,0.00572],3:[-0.38239,-0.95773,-0.00161],4:[-0.37993,-1.35003,0.00447]},
    103:{1:[-0.19819,-0.95817,-0.00195],2:[-0.19556,-1.35045,0.00445],3:[0.19362,-0.95622,-0.00191],4:[0.19632,-1.34792,0.00488]},
    104:{1:[0.77352,-1.34842,0.00806],2:[0.77129,-0.95523,0.00211],3:[0.38093,-1.34816,0.00568],4:[0.38023,-0.95594,-0.00119]},
    105:{1:[1.35231,-0.77031,0.00857],2:[1.35102,-0.37755,0.00485],3:[0.96006,-0.76920,0.00349],4:[0.95755,-0.37818,-0.00115]}
}

In [91]:
# get the supermodule in the room 

sm = my_room.sms[0]
survey_data = {'bottom':bottom_survey_data, 'top':top_survey_data}[sm.name]
serial_offset = {'bottom':0, 'top':85}[sm.name]

# look at RMS of deviations of the target positions from survey from the design positions

sum2 = 0.
for j_mpmt, mpmt in enumerate(sm.mpmts):
    i_mpmt = j_mpmt + serial_offset
    
    #fiducials_design = mpmt.get_fiducials('design', sm) # we don't know the location of the SM in the room
    fiducials_design = mpmt.get_fiducials('design', my_room) # or my_room?
    for i in range(4):
        survey_xyz = survey_data.get(i_mpmt,{}).get(i+1,None)
        if survey_xyz is not None:
            survey_location = np.array(survey_xyz)*1000. # convert to mm
            diff = np.subtract(survey_location,fiducials_design[i])
            sum2 += np.dot(diff,diff)
rms = np.sqrt(sum2/(4*len(sm.mpmts)))
print('RMS of deviations of the true fiduical markers compared to design positions: %.2f mm' % rms)

RMS of deviations of the true fiduical markers compared to design positions: 7.15 mm


In [92]:
# Combine the survey data to deduce the placement of the sm in the room (placement defined by 6 parameters: 3 locations and 3 rotations)
# then use the survey data for each mpmt to deduce its placement within the sm

# since some of the fiducials might not be measured in the actual survey, do not use their average for an mpmt to characterize the mpmt location
# instead, calculate the RMS of all fiducials: first assuming mpmt are placed according to design (to find sm placement) then with sm placement do each mpmt
# check if an additional iteration helps

# Start with model that sets all estimated placements (for the sm wrt room and the mpmts wrt sm) to their design values: execute this to start over

sm.place_est = sm.place_design.copy()
for mpmt in sm.mpmts:
    mpmt.place_est = mpmt.place_design.copy()

In [93]:
# Calculate sum of squares of residuals for all fiducials in sm 

def calc_dev2_all():
    sum2 = 0.
    n = 0
    for j_mpmt, mpmt in enumerate(sm.mpmts):
        i_mpmt = j_mpmt + serial_offset
        fiducials_est = mpmt.get_fiducials('est')
        for i in range(4):
            survey_xyz = survey_data.get(i_mpmt,{}).get(i+1,None)
            if survey_xyz is not None:
                survey_location = np.array(survey_xyz)*1000.
                diff = np.subtract(fiducials_est[i],survey_location)
                sum2 += np.dot(diff,diff)
                n += 1
    return n, sum2

# Calculate sum of squares of residuals for the fiducials in the fit mpmt

fit_mpmt = 1

def calc_dev2_one():
    sum2 = 0.
    n = 0
    fiducials_est = sm.mpmts[fit_mpmt].get_fiducials('est')
    for i in range(4):
        survey_xyz = survey_data.get(fit_mpmt+serial_offset,{}).get(i+1,None)
        if survey_xyz is not None:
            survey_location = np.array(survey_xyz)*1000.
            diff = np.subtract(fiducials_est[i],survey_location)
            sum2 += np.dot(diff,diff)
            n += 1
    return n, sum2

# parameters specify the sm placement
def calc_dev2_sm(params):
    loc = [params[i] for i in range(3)]
    rot_angles = [params[i] for i in range(3,6)]
    sm.place_est['loc'] = loc
    sm.place_est['rot_angles'] = rot_angles
    
    n, sum2 = calc_dev2_all()
    return sum2

# parameters specify the fit_mpmt placement
def calc_dev2_mpmt(params):
    loc = [params[i] for i in range(3)]
    rot_angles = [params[i] for i in range(3,6)]
    sm.mpmts[fit_mpmt].place_est['loc'] = loc
    sm.mpmts[fit_mpmt].place_est['rot_angles'] = rot_angles
    
    n, sum2 = calc_dev2_one()
    return sum2


In [97]:
# calculate x,y,z separately:
sum2 = [0.,0.,0.]
n = 0
for j_mpmt, mpmt in enumerate(sm.mpmts):
    i_mpmt = j_mpmt + serial_offset
    fiducials_est = mpmt.get_fiducials('est')
    for i in range(4):
        survey_xyz = survey_data.get(i_mpmt,{}).get(i+1,None)
        if survey_xyz is not None:
            survey_location = np.array(survey_xyz)*1000.
            diff = np.subtract(fiducials_est[i],survey_location)
            for j in range(3):
                sum2[j] += diff[j]*diff[j]
            n += 1
print(np.sqrt(sum2[0]/n),np.sqrt(sum2[1]/n),np.sqrt(sum2[2]/n))

0.9163434130786052 0.5959444090882065 0.20910889077604544


In [94]:
n, sum2 = calc_dev2_all()
print('all:',np.sqrt(sum2/n))

n, sum2 = calc_dev2_one()
print('one:',np.sqrt(sum2/n))

all: 7.726712323707468
one: 6.75672442830104


In [95]:
# find optimal sm placement parameters - start from the current values stored in place_est
start = sm.place_est['loc'] + sm.place_est['rot_angles']
result = optimize.minimize(calc_dev2_sm, start)
print(result.message)
    
print('est:', sm.get_placement('est'))

n, sum2 = calc_dev2_all()
print('rms of residuals:', np.sqrt(sum2/n))

Desired error not necessarily achieved due to precision loss.
est: {'location': [-1.1303429584290283, 0.806735476588154, -31.401922422970948], 'direction_x': [0.9999993630725105, 0.00031664034535177423, 0.001083325188981048], 'direction_z': [-0.0010832101117545553, -0.0003633778026970611, 0.9999993473060002]}
rms of residuals: 7.422603579001074


In [96]:
# find optimal mpmt placement parameters - start from the current values stored in place_est

for fit_mpmt in range(len(sm.mpmts)):

    start = list(sm.mpmts[fit_mpmt].place_est['loc']) + list(sm.mpmts[fit_mpmt].place_est['rot_angles'])
    result = optimize.minimize(calc_dev2_mpmt, start)
    
    n, sum2 = calc_dev2_one()
    if n>1:
        rms = np.sqrt(sum2/n)
        if rms > 1.:
            print('mpmt', fit_mpmt, result.message)
            print('est:', sm.mpmts[fit_mpmt].get_placement('est'))       
            print('rms of residuals:', rms)

n, sum2 = calc_dev2_all()
print('rms all:',np.sqrt(sum2/n))

mpmt 2 Desired error not necessarily achieved due to precision loss.
est: {'location': [576.9744494525135, 577.8840414670686, -33.55837627485508], 'direction_x': [-0.9999716111853307, 0.0026736475543646075, -0.007044745003821545], 'direction_z': [0.007064976999503191, 0.0076347276167803135, -0.9999458970535428]}
rms of residuals: 1.2669854672543877
mpmt 4 Desired error not necessarily achieved due to precision loss.
est: {'location': [-578.1036526912036, 577.90442340419, -32.388008065085074], 'direction_x': [0.999959088644003, -0.00029279935940962836, -0.009040758087159482], 'direction_z': [-0.009038434864003199, 0.00714743909729123, -0.9999336082008443]}
rms of residuals: 1.074185064460581
mpmt 5 Desired error not necessarily achieved due to precision loss.
est: {'location': [-578.7486781047179, 0.7158913559112966, -35.02640016377603], 'direction_x': [0.9999663048726825, -0.00020730335551893893, -0.00820646967899958], 'direction_z': [-0.00820593817747863, 0.002446358729527048, -0.9999

In [84]:
# Show in 3D the deviations between est and survey (or between est and true) to illustrate how survey optimization proceeds 

# to keep mpmt numbering consistent with the full WCTE detector, we need to add an offset
first_mpmt = {'bottom':0, 'barrel':21, 'top':85}[sm.name]

plot = k3d.plot()

origins = []
survey_vecs = []

survey_scale = 100. # scale up the length of deviations
color_survey = 0xff0000

color_mpmt = 0xabb2b9
n_point_mpmt = 8
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])

n_fd_point_mpmt = 40
fd_indices_mpmt = []
for i in range(n_fd_point_mpmt):
    fd_indices_mpmt.append([i,(i+1)%n_fd_point_mpmt])
    
n_point_ft = 20 # for feedthroughs and C holes
indices_ft = []
for i in range(n_point_ft):
    indices_ft.append([i,(i+1)%n_point_ft])

# draw the extent of the mpmt baseplates
for i_mpmt,mpmt in enumerate(sm.mpmts):
    
    p = mpmt.get_placement('est')
    location, direction_x, direction_z = p['location'], p['direction_x'], p['direction_z']

    baseplate_points = np.array(mpmt.get_xy_points('design'),dtype=np.float32)
    color = color_mpmt

    indices = indices_mpmt
    if mpmt.kind == 'FD':
        indices = fd_indices_mpmt
        
    plt_baseplate = k3d.lines(baseplate_points, indices, indices_type='segment', color=color)
    plot += plt_baseplate
    
    if mpmt.kind != 'FD':
        feedthrough_points = np.array(mpmt.get_xy_points('design', feature='feedthrough'),dtype=np.float32)
        plt_feedthrough = k3d.lines(feedthrough_points, indices_ft, indices_type='segment', color=color_mpmt)
        plot += plt_feedthrough
    else:
        feedthrough1_points = np.array(mpmt.get_xy_points('design', feature='fd_feedthrough1'),dtype=np.float32)
        plt_feedthrough1 = k3d.lines(feedthrough1_points, indices_ft, indices_type='segment', color=color_mpmt)
        plot += plt_feedthrough1
        feedthrough2_points = np.array(mpmt.get_xy_points('design', feature='fd_feedthrough2'),dtype=np.float32)
        plt_feedthrough2 = k3d.lines(feedthrough2_points, indices_ft, indices_type='segment', color=color_mpmt)
        plot += plt_feedthrough2

    # k3d complains about the following not being float32!
    plt_text = k3d.text(str(i_mpmt+first_mpmt), position=location, reference_point='cc', size=1., label_box=False, color=color_mpmt)
    plot += plt_text

    # draw the fiducial points
    fiducial_points = np.array(mpmt.get_fiducials('est'),dtype=np.float32)
    for i_f, fiducial_point in enumerate(fiducial_points):
        text_location = [fiducial_point[j] - 10.*direction_z[j] for j in range(3)]
        plt_text = k3d.text('f'+str(i_f+1), position=text_location, reference_point='cc', size=1., label_box=False, color=color_mpmt)
        plot += plt_text

    fiducial_locations = np.array(fiducial_points, dtype=np.float32)
    plt_fps = k3d.points(positions=fiducial_locations,
                        point_size=8.,
                        shader='3d',
                        color=color_mpmt)
    plot += plt_fps

    # show the surveyed locations of the fiducial points
    fiducial_survey = []
    for i in range(4):
        survey_xyz = survey_data.get(i_mpmt+first_mpmt,{}).get(i+1,None)
        if survey_xyz is not None:
            fiducial_survey.append(np.array(survey_xyz)*1000.)
        
    fiducial_survey = np.array(fiducial_survey, dtype=np.float32)
    plt_fs = k3d.points(positions=fiducial_survey,
                        point_size=8.,
                        shader='3d',
                        color=color_survey)
    plot += plt_fs
    
    # show a vector from the estimated to the surveyed
    fiducials_est = mpmt.get_fiducials('est')
    for i in range(4):
        survey_xyz = survey_data.get(i_mpmt+first_mpmt,{}).get(i+1,None)
        if survey_xyz is not None:
            survey_location = np.array(survey_xyz)*1000.
            
            origins.append(fiducials_est[i])
            diff = np.subtract(survey_location,fiducials_est[i])
            scaled_diff = list(diff*survey_scale)
            survey_vecs.append(scaled_diff)

plt_survey_vecs = k3d.vectors(origins=origins, vectors=survey_vecs, color=color_survey, head_size=250.)
plot += plt_survey_vecs

plot.display()

Output()